# Логистическая регрессия#

***Проверка качества классификатора***

In [1]:
from nltk.stem.snowball import SnowballStemmer
import csv
import re
from nltk.corpus import stopwords
import pymorphy2
from nltk.stem.porter import *
from sklearn.metrics import classification_report
import numpy as np
import pickle 

stops = set(stopwords.words("english")) | set(stopwords.words("russian"))

morph=pymorphy2.MorphAnalyzer()
stemmer=SnowballStemmer('russian')


In [2]:
def treatment_text(review):
    review_text = re.sub("[^а-яА-Яa-zA-Z0-9]", " ", review)
    words = review_text.lower().split()
    words = [w for w in words if not w in stops]
    words = [morph.parse(w)[0].normal_form for w in words]
    words = [stemmer.stem(w) for w in words]
    words = [w for w in words if not w in stops]
    return(words)

In [3]:
# создание dataset/x-статья,y-tag(0,1)
# dataset = [sentence_origin,sentence_tr,tag,kws,[kw_in_sentence]]
dataset = []
file='articles.csv'

with open(file,'r', encoding='utf-8',newline='') as f:
    reader = csv.reader(f,delimiter=',')
    for row in reader:            
        if row!=[]:
            kws=''.join(row[0])
            text = ''.join(row[2])
            sentences_origin = text.split('.')
            sentences_tr = list(map(treatment_text,sentences_origin))
            kws_tr = treatment_text(kws)  
            kws_tr_set=set(kws_tr)  
            for sentence_tr,sentence_or in zip(sentences_tr,sentences_origin):
                sentence_tr_set=set(sentence_tr)
                if len(sentence_tr)>5 and kws_tr_set.intersection(sentence_tr_set):
                    dataset.append((sentence_or,' '.join(sentence_tr),'1',kws,' '.join(kws_tr_set.intersection(sentence_tr_set))))
                elif len(sentence_tr)>5:
                    dataset.append((sentence_or,' '.join(sentence_tr),'0',kws,'None'))


In [4]:
with open('cls_model.pickle', 'rb') as f:
    cls = pickle.load(f)

[nltk_data] Downloading package punkt to C:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to C:\nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     C:\nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [5]:
with open('tf_idf_model.pickle',  'rb') as f:
    tfidf = pickle.load(f)

In [6]:
x_true = [dataset[i][1] for i in range(len(dataset))]
y_true = [dataset[i][2] for i in range(len(dataset))]

In [7]:
pred=cls(tfidf(x_true))

In [8]:
y_true = list(map(lambda x: int(x[0]),y_true))
pred = list(map(lambda x: int(x[0]),pred))
print(classification_report(y_true, pred, labels=[0, 1]))


              precision    recall  f1-score   support

           0       0.61      0.85      0.71      2224
           1       0.80      0.52      0.63      2558

    accuracy                           0.68      4782
   macro avg       0.71      0.69      0.67      4782
weighted avg       0.71      0.68      0.67      4782



**Проверка качества выделения ключевых слов**

In [22]:
from nltk.stem.snowball import SnowballStemmer
import csv
import re
import pymorphy2
from nltk.stem.porter import *
import pickle
from deeppavlov.dataset_readers.basic_classification_reader import BasicClassificationDatasetReader
from deeppavlov.dataset_iterators.basic_classification_iterator import BasicClassificationDatasetIterator
from deeppavlov.dataset_iterators.basic_classification_iterator import BasicClassificationDatasetIterator
from deeppavlov.models.tokenizers.nltk_moses_tokenizer import NLTKMosesTokenizer
from deeppavlov.core.data.simple_vocab import SimpleVocabulary
from deeppavlov.models.sklearn import SklearnComponent
from deeppavlov.metrics.accuracy import sets_accuracy
import numpy as np

from nltk.corpus import stopwords
from google.oauth2 import service_account

# credentials = service_account.Credentials.from_service_account_file('./diplom-275218-a097a7baee3f.json')
# project_id = 'diplom-275218'
# private_key = 'diplom-275218-a097a7baee3f.json'
stops = set(stopwords.words("english")) | set(stopwords.words("russian"))
stops.add('рис')
stops.add('университет')
stops.add('брянск')

morph=pymorphy2.MorphAnalyzer()
stemmer=SnowballStemmer('russian')


dict_stop=set(['метод','определение','условие','момент','значение','результат','критерий',
               'работа','вариант','брянский государственный университет','научнотехнический вестник',
              'соответствие','такой образ','весь критерий','пример','выбор','ключевое слово','период',
              'уравнение','формула','множитель','повышение','оценка','проведение',
              'машина','нагрузка','брянская область','точка','случай','расчет','таблица','расчёт',
              'с показатель','град','обработка','статья','элемент','раз','применение','центр','форма','важная задача'])

path = 'C:/Users/Ирина/Desktop/keywords'

with open(path+'/cls_model.pickle', 'rb') as f:
    cls = pickle.load(f)

with open(path+'/tf_idf_model.pickle', 'rb') as f:
    tfidf = pickle.load(f)

def treatment_text2(text):
    text = re.sub("[^а-яА-Яa-zA-Z0-9,.?]", " ", str(text))
    text = text.replace('\t',' ')
    text = text.replace('\n', ' ')
    while text.find('  ')!=-1:
        text = text.replace('  ',' ')
    text = str(text)
    return text

def treatment_text(review):
    review_text = re.sub("[^а-яА-ЯЁёa-zA-Z0-9]", " ", review)
    review_text = review_text.replace('ё','е')
    review_text = review_text.replace('Ё', 'Е')
    words = review_text.lower().split()
    words = [w for w in words if not w in stops]
    words = [morph.parse(w)[0].normal_form for w in words]
    words = [stemmer.stem(w) for w in words]
    words = [w for w in words if not w in stops]
    return(words)


def normalize(keyphrase):
    try:
        words = keyphrase.split()
        if len(words)>1:
            main_noun = words[-1]
            flag_noun = False
            flag_adj = False
            for i in words[:-1]:
                if 'NOUN' not in  morph.parse(i)[0].tag and 'ADJF' not in morph.parse(i)[0].tag:
                    return morph.parse(words[0])[0].lexeme[0][0] + ','+morph.parse(words[2])[0].lexeme[0][0]
                if 'NOUN' in morph.parse(i)[0].tag:
                    flag_noun=True
                if 'ADJF' in morph.parse(i)[0].tag:
                    flag_adj=True
            s = ''

            if flag_noun is False:
                for word in words[:-1]:
                    for i in morph.parse(word)[0].lexeme:
                        if morph.parse(main_noun)[0].tag.gender in i.tag and 'nomn' in i.tag:

                            s+=i.word + ' '
                            break
                return s + morph.parse(main_noun)[0].lexeme[0][0]

            elif flag_noun is True and flag_adj is False:
                s = morph.parse(words[0])[0].lexeme[0][0]
                for i in words[1:]:
                    try:
                        s+= ' '+morph.parse(i)[0].lexeme[1][0]
                    except:
                        pass
                return s

            else:
                flag_first_noun=False

                for word in words:
                    if 'ADJF' in  morph.parse(word)[0].tag and flag_first_noun is False:
                        rod = find_rod(words[words.index(word)+1:])
                        for i in morph.parse(word)[0].lexeme:
                            if rod in i.tag and 'nomn' in i.tag:

                                s+=i.word + ' '
                                break
                    elif 'ADJF' in  morph.parse(word)[0].tag and flag_first_noun is True:
                        rod = find_rod(words[words.index(word)+1:])
                        for i in morph.parse(word)[0].lexeme:
                            if rod in i.tag and 'gent' in i.tag:
                                s+=i.word + ' '
                                break
                    elif  'NOUN' in  morph.parse(word)[0].tag and flag_first_noun is False:
                        flag_first_noun=True

                        s += morph.parse(word)[0].lexeme[0][0] + ' '
                    else:

                        s += morph.parse(word)[0].lexeme[2][0] +' '


            return s
        elif len(words)==1:
            return morph.parse(words[0])[0].lexeme[0][0]
    except:
        pass
#         print(' '.join(words),'НЕ МОЖЕТ БЫТЬ ПРЕОБРАЗОВАН')

def find_rod(words):
    for w in words:
        if 'NOUN' in morph.parse(w)[0].tag:
            return morph.parse(w)[0].tag.gender


def find_adj(sentence, index_noun, pred_noun):
    if pred_noun == 0 and index_noun == 0:
        return 0
    if pred_noun == 0 and index_noun != 0:
        i = index_noun - 1
        flag_ad = False
        flag_noun = False
        while i >= 0:
            if 'ADJF' in morph.parse(sentence[i])[0].tag:
                flag_ad = True
            elif 'NOUN' in morph.parse(sentence[i])[0].tag:
                flag_noun = True
            elif flag_ad is False and flag_noun is False:
                return i + 1

            flag_ad = False
            flag_noun = False
            i -= 1
        return i + 1

    else:
        i = index_noun - 1
        flag_ad = False
        flag_noun = False
        while i > pred_noun:
            if 'ADJF' in morph.parse(sentence[i])[0].tag:
                flag_ad = True
            elif 'NOUN' in morph.parse(sentence[i])[0].tag:
                flag_noun = True
            elif flag_ad is False and flag_noun is False:
                return i + 1

            flag_ad = False
            flag_noun = False
            i -= 1
        return i + 1


def keyword_extraction(text):
    sentences_origin = text.split('.')
    sentences_tr = list(map(treatment_text, sentences_origin))
    test=[]
    for sentence_tr, sentence_or in zip(sentences_tr, sentences_origin):
        # sentence_tr_set = set(sentence_tr)
        # if len(sentence_tr) > 5 and kws_tr_set.intersection(sentence_tr_set):
        #     test.append((sentence_or, ' '.join(sentence_tr)))
        if len(sentence_tr) > 5:
            test.append((sentence_or, ' '.join(sentence_tr)))
    x_test = []
    for s in test:
        x_test.append(s[1])
#     x_test = tuple(x_test)
#     y_pred = model.predict(tokenizer.texts_to_matrix(x_test, mode='binary'))
#     print(y_pred,x_test)



    y_pred2 = cls(tfidf(x_test))
    

    sentences_with_kw = []
    for i in range(len(y_pred2)):
        if y_pred2[i] == '1':
#             print(test[i])
            sentences_with_kw.append(test[i])
    dict_cand = {}
    for sentence in sentences_with_kw:
        cand = extract_candidate(sentence[0], sentences_with_kw)
        cand = list(map(normalize, cand))
        for key_cand in cand:
            if key_cand in dict_cand:
                dict_cand[key_cand] += 1
            else:
                dict_cand[key_cand] = 1
    dict_cand_sort = {k: v for k, v in sorted(dict_cand.items(), key=lambda item: item[1], reverse=True)}
    result = []
    count = 0
    k = set(['1', '2', '3', '4', '5', '&', '?', '/', '\\', '!'])

    for word in dict_cand_sort.keys():
        try:
            if count < 10:
                if ',' in word:
                    word = word.split()
                    result += word
                elif k.intersection(set(word)) == set() and word not in dict_stop and 'ADJF' not in morph.parse(word)[
                    0].tag:
                    result.append(word)
                    count += 1
            else:
                break
        except:
            pass
    for i in result:

        flag = False
        s = i.replace(',', ' ')
        s = s.split(' ')
        for j in s:
            if 'NOUN' in morph.parse(j)[0].tag:
                flag = True
        if flag is False:
            result.remove(i)
    result = list(map(lambda x: x.replace(',', ' '), result))
    result = ', '.join(result)
    if result!='' :
        return result
#     else:
#         keywords = r.keywords_extract(text)
#         keywords = ', '.join(keywords).capitalize()
#         return keywords
def extract_candidate(origin, sentences_with_kw):
    candidate = []
    x = re.sub("[^а-яА-Я]", " ", origin)
    while x.find('  ') != -1:
        x = x.replace('  ', ' ')
    x = x.split()
    x = list(filter(lambda x: len(x) > 2, x))
    pr = 0

    for i in range(len(x) - 1):
        if 'NOUN' in morph.parse(x[i])[0].tag and 'NOUN' in morph.parse(x[i + 1])[0].tag and i + 2 != len(x):
            pass
        elif 'NOUN' in morph.parse(x[i])[0].tag and 'NOUN' in morph.parse(x[i + 1])[0].tag and i + 2 == len(x):
            try:
                k = find_adj(x, i + 2, pr)
                pr = i
                if ' '.join(x[k:i + 1]) in sentences_with_kw[2][0]:
                    candidate.append(' '.join(x[k:i + 2]))
                else:
                    begin = origin.find(x[k])
                    end = origin.find(x[i + 1]) + len(x[i + 1])
                    candidate += origin[begin:end].split(',')
            except:
                pass
        elif 'NOUN' in morph.parse(x[i])[0].tag and 'NOUN' not in morph.parse(x[i + 1])[0].tag:

            k = find_adj(x, i, pr)
            pr = i

            if ' '.join(x[k:i + 1]) in origin:
                candidate.append((' '.join(x[k:i + 1])))
            else:
                begin = origin.find(x[k], len(' '.join(x[:k])))
                end = origin.find(x[i], len(' '.join(x[:i]))) + len(x[i])
                candidate += origin[begin:end].split(',')
    return candidate

In [31]:
# создание dataset/x-статья,y-tag(0,1)
# dataset = [sentence_origin,sentence_tr,tag,kws,[kw_in_sentence],text]
result = []
file='test_sample_kws.csv'
k=0

with open(file,'r', encoding='cp1251',newline='') as f:
    reader = csv.reader(f,delimiter=';')
    for row in reader:
        if row!=[]:
#             print(k)
            k+=1
            kws_true =''.join(row[0])
            ann = ''.join(row[1])
            text = ''.join(row[2])
            if text!='':
                kws_pred = keyword_extraction(text)
                result.append([kws_true,kws_pred])
            
                
                

In [32]:
len_true = []
len_pred = []
len_intersection = []

for i in range(len(result)-1):
    kws_true = set(treatment_text(result[i][0])) 
    kws_pred = set(treatment_text(result[i][1])) 
    len_true.append(len(kws_true))
    len_pred.append(len(kws_pred))
    len_intersection.append(len(kws_true.intersection(kws_pred)))
#     print(result[i][0])
#     print(result[i][1])
#     print(len(kws_true.intersection(kws_pred)))


recall =sum(len_intersection)/sum(len_true)
precision = sum(len_intersection)/sum(len_pred)
print(f'precision    {precision}')
print(f'recall       {recall}')
print(f'f_measure    {2*precision*recall/(precision+recall)}')

precision    0.32318104906937395
recall       0.4692874692874693
f_measure    0.3827655310621243


**Обучение модели**

In [ ]:
# создание dataset/x-статья,y-tag(0,1)
# dataset = [sentence_origin,sentence_tr,tag,kws,[kw_in_sentence]]
dataset = []

files=['./train_log/articles2019.csv','./train_log/articles2018.csv','./train_log/articles2017.csv']
for file in files:
    with open(file,'r', encoding='utf-8',newline='') as f:
        reader = csv.reader(f,delimiter=',')
        for row in reader:     
            if row!=[]:
                kws=''.join(row[0])
                text = ''.join(row[2])
                sentences_origin = text.split('.')
                sentences_tr = list(map(treatment_text,sentences_origin))
                kws_tr = treatment_text(kws)  
                kws_tr_set=set(kws_tr)        
                for sentence_tr,sentence_or in zip(sentences_tr,sentences_origin):
                    sentence_tr_set=set(sentence_tr)
                    if len(sentence_tr)>5 and kws_tr_set.intersection(sentence_tr_set):
                        dataset.append((sentence_or,' '.join(sentence_tr),'1',kws,' '.join(kws_tr_set.intersection(sentence_tr_set))))
                    elif len(sentence_tr)>5:
                        dataset.append((sentence_or,' '.join(sentence_tr),'0',kws,'None'))



In [ ]:
# прочитать dataset из csv файла
dr = BasicClassificationDatasetReader().read(
    data_path=k,
    train='dataset.csv',
    x = 'text',
    y = 'tag',
)

In [ ]:
# initialize data iterator splitting `train` field to `train` and `valid` in proportion 0.8/0.2
train_iterator = BasicClassificationDatasetIterator(
    data=dr,
    field_to_split='train',  # field that will be splitted
    split_fields=['train', 'valid'],   # fields to which the fiald above will be splitted
    split_proportions=[0.8, 0.2],  #proportions for splitting
    split_seed=23,  # seed for splitting dataset 23
    seed=42)  # seed for iteration over dataset 42

In [ ]:
tokenizer = NLTKMosesTokenizer()
train_x_lower_tokenized = tokenizer(train_iterator.get_instances(data_type='train')[0])

In [ ]:
# initialize simple vocabulary to collect all appeared in the dataset classes
classes_vocab = SimpleVocabulary(
    save_path='.classes.dict',
    load_path='./classes.dict')
classes_vocab.fit((train_iterator.get_instances(data_type='train')[1]))
classes_vocab.save()

In [ ]:
# also one can collect vocabulary of textual tokens appeared 2 and more times in the dataset
token_vocab = SimpleVocabulary(
    save_path='./tokens.dict',
    load_path='./tokens.dict',
    min_freq=2,
    special_tokens=('<PAD>', '<UNK>',),
    unk_token='<UNK>')
token_vocab.fit(train_x_lower_tokenized)
token_vocab.save()
token_vocab.freqs.most_common()[:10]

In [ ]:
# initialize TF-IDF vectorizer sklearn component with `transform` as infer method
tfidf = SklearnComponent(
    model_class="sklearn.feature_extraction.text:TfidfVectorizer",
    infer_method="transform",
    save_path='./tfidf_v0.pkl',
    load_path='./tfidf_v0.pkl',
    mode='train')
tfidf.fit(train_iterator.get_instances(data_type='train')[0])
tfidf.save()

In [ ]:
# get all train and valid data from iterator
x_train, y_train = train_iterator.get_instances(data_type="train")
x_valid, y_valid = train_iterator.get_instances(data_type="valid")


In [ ]:
# initialize sklearn classifier, all parameters for classifier could be passed
cls = SklearnComponent(
    model_class="sklearn.linear_model:LogisticRegression",
    infer_method="predict",
    save_path='./logreg_v0.pkl',
    load_path='./logreg_v0.pkl',
    C=1,
    mode='train')

In [ ]:
# fit sklearn classifier and save it
cls.fit(tfidf(x_train), y_train)
cls.save()